[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nkeriven/ensta-mt12/blob/main/notebooks/05_clustering/N4_EM_basic.ipynb)

# EM basic example 
The purpose of this labwork is to implement a Gaussian Mixture Model Clustering algorithm, using Expectation Maximization (EM) method. First, a code is proposed on a 1D example implementing directly the theoretical formula from the lecture. Second, the obtained results are compared with the results obtained using sklearn GMM function. 

## Data import from matlab file

In [ ]:
!wget https://raw.githubusercontent.com/nkeriven/ensta-mt12/main/notebooks/data/fictitious_train.mat -O fictitious_train.mat

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.stats as stats
%matplotlib inline

Data_train = loadmat('fictitious_train.mat')
#Data_train.keys()

X=Data_train.get('Xtrain')
#H=Data_train.get('__header__')
#print("dimensions ox X ={}".format(X.shape))

## intialization of parameters and Kernel computation
- note that here, the number of clusters is set a priori 

In [ ]:
#EM
N=X.shape[0]
K=2
p=1
MaxIter=100
#init 
init=np.random.choice(range(N), size=K, replace=False)
pivec= np.ones((K,p))/K 
muvec= np.zeros((K,p)) 
sigvec= np.zeros((K,p))
postpr=np.zeros((N,K))

for k in range(K):
    muvec[k,:]=X[init[k],:]  #different means
    sigvec[k,:]=np.var(X)/K #say, global variance divided by two

### Exercize
- Identify the arrays above wrt to the charateristics of the GMM introduced in the lecture
- Explain why different means are initialized, whereas same variances may be used
- Comment the line codes below, briefly

In [ ]:
#A posteriori Proba to be in a class

for t in range (0,MaxIter):
    #E-Step
    for i in range(0,N): 
        for k in range(K):
            postpr[i,k]=pivec[k]* stats.norm.pdf( X[i], muvec[k,:],  np.sqrt( sigvec[k,:] ) )
        postpr[i,:] /= postpr[i,:].sum()
            
    #M-step    
    for k in range (0,K):
        S = np.sum(postpr[:,k])
        pivec[k,:]= S/N
        muvec[k,:]= postpr[:,k].dot(X) / S
        sigvec[k,:]=postpr[:,k].dot((X-muvec[k,:])**2) / S


print(f'muvec={muvec}')
print(f'sigvec={sigvec}')
print(f'pivec={pivec}')


### Plot
Below, the plot displays the 2 Gaussian pdfs involved in the mixture; the total pdf; the histogram of the data. 

In [ ]:
Xt=np.linspace(-2,8,1000)
g = np.zeros_like(Xt)
for k in range(K):
    gmix = pivec[k]*stats.norm.pdf(Xt,muvec[k],np.sqrt(sigvec[k]))
    g += gmix
    plt.plot(Xt,gmix,label=f'g{k}')
plt.plot(Xt, g, label='full density')
plt.legend()
plt.xlabel('X');
plt.hist(X, density=True)

### Exercize
Increase K. Play with the initialization a bit. Comment.

## Sklearn implementation

In [ ]:

from sklearn.mixture import GaussianMixture
# Try GMMs using full covariance (no constraints imposed on cov)
est = GaussianMixture(n_components=K, 
                      covariance_type='full', 
                      max_iter= MaxIter,
                      random_state=0)

est.fit(X)

print(f'est.cov={est.covariances_}')
print(f'est.means={est.means_}')
print(f'est.weights={est.weights_}')

### Exercize 
- compare the results obtained with Sklearn with the previously obtained results. 
- Comments? 
- Add the mixture pdf to the plot

In [ ]:
Xt=np.linspace(-2,8,1000)
gsk = np.zeros_like(Xt)
for k in range(K):
    gskmix=est.weights_[k]*stats.norm.pdf(Xt,np.squeeze(est.means_[k]),
                  np.sqrt(np.squeeze(est.covariances_[k])))
    gsk += gskmix
    plt.plot(Xt,gskmix, label=f'gsk{k}')
                         
plt.plot(Xt, gsk, label='full density')
plt.legend()
plt.xlabel('X');
plt.hist(X, density=True)

In [ ]:
from sklearn.cluster import KMeans

#Kmeans vs EM
g = np.zeros_like(Xt)
gmix = dict()
for k in range(K):
    gmix[k] = pivec[k] * stats.norm.pdf(Xt,muvec[k],np.sqrt(sigvec[k]))
    g += gmix[k]
for k in range(K):
    resp =  gmix[k] / g ;
    plt.plot(Xt,resp,label=f'responsability class{k}')
#plt.plot(Xt,resp1,label='responsability class1')

est_km = KMeans(n_clusters = K, init = 'k-means++')
Y=est_km.fit_predict(X)
for k in range(K):
    plt.scatter(X[Y==k],np.ones_like(X[Y==k])*.5,marker='+',
                label=f'class{k}')
#plt.legend();

### Exercize 
- Compare and interpret Kmeans wrt EM; 
- Interpret the responsability functions resp0 and resp1. 